##### Justin Valdez 

# The Battle of Neighborhoods 

# Applied Data Science Capstone by IBM/Coursera
__________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
### Table of contents
* [Introduction/Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction/Business Probelm: <a name="introduction"></a>
__________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

### 1. Background
<u>**New York City (NYC)**</u>, The City, is the most poulous city in the United States. NYC has an estimated poulation of 8,398,749 distrubuted over about 302.6 square miles in 2018.

NYC is located at the southern tip of the U.S. state of New york, the city is the center of the New York metropitan area. With the amount of people in the metropolitan statistical area, NYC is one of the world's most populous megacities. Therefore, NYC is best described as the cultural, financial, and media capital of the world. The city influences commerce, entertainment, research, technology, education, politices, tourism, art, fashion, and sports.

NYC is set on one of the world's largest natural harbors, so it is composed of five boroughs - Brooklyn, Queens, Manhattan, the Bronx, and Staten Island - in which each is a county of the State of New York

### Objective
The aim of this report is to study and analyze the neighborhoods of New York City, group them into similar clusters and analyze those clusters to gather meaningful information. The data for New York has been touched upon on previous projects, but will be further finished.

### Target Audience:
The information provided by this project would be valuable for people who are interested or considering in relocating to different parts of the world. 

## 2. Problem Description
__________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
Now let me explain the context of this Capstone project through a scenario that can better be understood. Say you're preparing to move out from your parent's or guardian's home. You love your family, loved ones, and friends, but it's now time for you to move on to greater things and experience new ways of life. 

So, say you receive a job offer from a great company in *Manhattan, New York*. The company is offering a great pay and benefits. You have to decide to where in Manhattan you will stay and start your new life.

### 2.1 Data Description <a name="data"></a>
Considering the objective stated above, we can list the data sources used for the analysis below.

* New York City Data
  - The following Catalog Data page was scraped to pull out the necessary information: https://data.cityofnewyork.us/api/views/swpk-hqdp/rows.csv?accessType=DOWNLOAD
https://geo.nyu.edu/catalog/nyu_2451_34572


* Coordinate Data for each Neighborhood
  - The following csv file was used to get the latitude and longitude for NYC's Neighborhoods https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=Original

### Installing and importing important libraries

In [79]:
# Libraries
print("Installing libraries . . . ")
print(". . .")
print("")

# Numpy and Pandas
import numpy as np # library used to handle data in a vectorized manner
import pandas as pd # pandas library
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# JSON
import json # library used to handle JSON files
import requests # library used to handle requests

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Time
import time # module provides various time-related functions

# Random 
import random # library for random number generation

# CSV
import csv # module implements classes to read and write tabular data in CSV format

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Seaborn
import seaborn as sns # data visualization library based on matplotlib

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Geopy
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# Folium
!pip install folium
import folium # plotting library
from folium import plugins # Wrap some of the most popular leaflet external plugins


# Success 
print("")
print("Libraries imported")

Installing libraries . . . 
. . .


Libraries imported


## 3. Methodology <a name="methodology"></a>
__________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

## Methodology Proccessing Seen Below
* Proccessing
  - Download and Explore Dataset
  - Transform Data Into Pandas Dataframe
  - Use Geopy Library in Order to get The Latitude and Longitude Values of New York City
  - Create Map of New York With Neighborhoods Superimposed on Top
  - Segmenting and Clustering Only the Neighborhoods in Manhattan
  - Retrieving the Geopgrpahical Coordinates of Manhattan
  - Visualization of Manhattan and the Neighborhoods
  
To start with our analysis, we use the json data and transform it into a pandas dataframe.

Kindly see below;

### 1. Download and Explore Dataset

In [80]:
# Download the dataset
# Source of dataset = https://geo.nyu.edu/catalog/nyu_2451_34572
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
    
# Load data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
print('Data loaded!')

Data loaded!


In [81]:
# Explore the data
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

### Transform Data Into *Pandas* Dataframe

In [82]:
# Variable including the list of the neighborhoods
neighborhoods_data = newyork_data['features']

# Define dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# Loop through the data and fill the dataframe one row at a time
# For loop
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

# Display dataframe
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [83]:
# Check if the dataset has correct number of boroughs and neighborhoods
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]))

The dataframe has 5 boroughs and 306 neighborhoods.


### Use Geopy Library in Order to get the *Latitude* and *Longitude* Values of New York City

In [84]:
# NYC Data
address = 'New York City, NY'

# Variables
geolocator = Nominatim(user_agent="ny_explorer")
location   = geolocator.geocode(address)
latitude   = location.latitude
longitude  = location.longitude

print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Create Map of New York With Neighborhoods Superimposed on Top

In [85]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius       = 5,
        popup        = label,
        color        = 'blue',
        fill         = True,
        fill_color   ='#3186cc',
        fill_opacity = 0.7,
        parse_html   = False).add_to(map_newyork)  
    
# Display Map  
map_newyork

### Segmenting and Clustering Only the Neighborhoods in Manhattan

In [86]:
# Manhattan variable
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop = True)

# Display first 5 rows
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


### Retrieving the Geopgrpahical Coordinates of Manhattan

In [87]:
# Variables
address = 'Manhattan, NY'
 
geolocator = Nominatim(user_agent = "ny_explorer")
location   = geolocator.geocode(address)
latitude   = location.latitude
longitude  = location.longitude

print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


### Visualization of Manhattan and the Neighborhoods

In [88]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start = 11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius       = 5,
        popup        = label,
        color        = 'blue',
        fill         = True,
        fill_color   = '#3186cc',
        fill_opacity = 0.7,
        parse_html   = False).add_to(map_manhattan)  

# Display Map
map_manhattan

## Utilizing Foursquare API to Explore the Neighborhoods
* Utilizing Foursquare API to Explore the Neighborhoods
  - Explore the First Neighborhood in the Dataframe
  - Retrieve the Latitude and Longitude Values
  - Finding the Top 100 Venues That Are in Marble Hill Within a Radius of 500 Meters
  - Send the GET Request and Examine the Results
  - Borrowing the get_category_type Function From the Foursquare Lab
  - How many venues were returned by Foursquare?

In [89]:
# The code was removed by Watson Studio for sharing.

### Explore the First Neighborhood in the Dataframe

In [90]:
# Name of the first neighborhood
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

### Retrieve the Latitude and Longitude Values

In [91]:
# Latitude and longitude values
neighborhood_latitude  = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name      = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


### Finding the Top 100 Venues That Are in Marble Hill Within a Radius of 500 Meters
Start utilizing the Foursquare API to explore the neighborhoods and segment them. We set the LIMIT parameter to 100, which would limit the number of venues returned by the Foursquare API and the radius of 500 meter.

In [92]:
# The code was removed by Watson Studio for sharing.

'https://api.foursquare.com/v2/venues/explore?&client_id=KLIUO4VJZIL5HAXTIX1WP2BK3BKREZ0DGBUURXSAWVBHZGFY&client_secret=14D4TXDXOFDSBSVQI3LCENYUWRA0LEFUNVND42J4T2D34QIC&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

### Send the GET Request and Examine the Results
Results contain venues in the Manhattan neighborhood.

In [93]:
# Results variable
results = requests.get(url).json()

# Display results
results

{'meta': {'code': 200, 'requestId': '5ebc9562be61c9001ba8f8b8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 26,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

### Borrowing the *get_category_type* Function From the Foursquare Lab

In [94]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# Clean JSON data and structure it into a pandas dataframe
    
venues        = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

# Display
nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin',Donut Shop,40.877136,-73.906666


### How many venues were returned by Foursquare?

In [95]:
# Print and Calculate how many venues were returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

26 venues were returned by Foursquare.


## Explore Each Neighborhood in Manhattan <a name="analysis"></a>
* Exploring 
  - Create a New Dataframe 
  - Check Size of the Resulting Dataframe 
  - Check How Many Venues Were Returned for Each Neighborhood
  - Determine How Many Unique Categories From the Venues can be Curated
  - New Dataframe Size

A function will be created to repeat the same process to all the neighborhoods in Manhattan.

In [96]:
# Retrieve nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    # Venues Dictionary
    venues_list = []
    
    # For Loop
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    # Display List
    return(nearby_venues)

### Create a New Dataframe 

In [97]:
# Results
print("Printing Dataframe Results")
print(". . .")
print("")

# Dataframe 
manhattan_venues = getNearbyVenues(names     = manhattan_data['Neighborhood'],
                                   latitudes = manhattan_data['Latitude'],
                                   longitudes= manhattan_data['Longitude'])

print("")
print("Results printed!")

Printing Dataframe Results
. . .

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards

Results printed!


### Check Size of the Resulting Dataframe 

In [98]:
# Display size
print("Size of dataframe")
print(manhattan_venues.shape)

# Display top 5 rows
manhattan_venues.head()

Size of dataframe
(3004, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


### Check How Many Venues Were Returned for Each Neighborhood

In [99]:
# Display results
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,59,59,59,59,59,59
Carnegie Hill,86,86,86,86,86,86
Central Harlem,45,45,45,45,45,45
Chelsea,97,97,97,97,97,97
Chinatown,100,100,100,100,100,100
Civic Center,87,87,87,87,87,87
Clinton,100,100,100,100,100,100
East Harlem,42,42,42,42,42,42
East Village,100,100,100,100,100,100


### Determine How Many Unique Categories From the Venues can be Curated

In [100]:
# Display unique categories
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 329 uniques categories.


## Analyze Each Neighborhood in Manhattan
* Analyze Each Neighborhood in Manhattan
  - Group Rows by Neighborhood and Take the Mean of the Frequency of Occurrence of Each Category
  - Confirm New Dataframe Size
  - Display Each Neighborhood With the Top 5 Most Common Venues
  - Put This Data Into a Pandas Dataframe
  
  
This part of the project, One Hot Encoding will be used, the data frame was standardized and the data was grouped by neighborhoods, we created a new data frame consisting of the top 10 venues in each neighborhood.

In [101]:
# One Hot Encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix = "", prefix_sep = "")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns    = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

# Display top 5 rows
manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaur

### New Dataframe Size

In [102]:
# Display size
print("Dataframe size")
manhattan_onehot.shape

Dataframe size


(3004, 330)

### Group Rows by Neighborhood and Take the Mean of the Frequency of Occurrence of Each Category

In [103]:
# Grouped variable
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

# Display grouped dataframe
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaur

### Confirm New Dataframe Size

In [104]:
# Display size
print("Dataframe size")
manhattan_grouped.shape

Dataframe size


(40, 330)

### Display Each Neighborhood With the Top 5 Most Common Venues

In [105]:
# Top 5 venues variable
num_top_venues = 5

# For Loop
for hood in manhattan_grouped['Neighborhood']:
    # Print neighborhood
    print("---- "  + hood + " ----")
    
    # Variables
    temp         = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp         = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp         = temp.round({'freq': 2})
    
    # Print values
    print(temp.sort_values('freq', ascending = False).reset_index(drop = True).head(num_top_venues))
    print('\n') # New line

---- Battery Park City ----
           venue  freq
0           Park  0.12
1          Hotel  0.07
2    Coffee Shop  0.05
3  Memorial Site  0.05
4            Gym  0.05


---- Carnegie Hill ----
                 venue  freq
0          Coffee Shop  0.08
1          Yoga Studio  0.03
2  Japanese Restaurant  0.03
3            Bookstore  0.03
4                 Café  0.03


---- Central Harlem ----
                 venue  freq
0   Chinese Restaurant  0.07
1   African Restaurant  0.07
2   Seafood Restaurant  0.04
3  American Restaurant  0.04
4       Cosmetics Shop  0.04


---- Chelsea ----
            venue  freq
0     Art Gallery  0.16
1     Coffee Shop  0.06
2            Café  0.04
3  Ice Cream Shop  0.03
4       Juice Bar  0.02


---- Chinatown ----
                venue  freq
0  Chinese Restaurant  0.08
1              Bakery  0.04
2        Cocktail Bar  0.04
3                 Spa  0.03
4         Coffee Shop  0.03


---- Civic Center ----
               venue  freq
0        Coffee Shop  0.06


### Put This Data Into a *Pandas* Dataframe

In [106]:
# Function 
# Sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories        = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Create New Dataframe 
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']

# For Loop 
for ind in np.arange(num_top_venues):
    # Conditions
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

# For Loop
for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

# Display top 10 venues
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Memorial Site,Coffee Shop,Gourmet Shop,Wine Shop,Food Court,Shopping Mall,Boat or Ferry
1,Carnegie Hill,Coffee Shop,Grocery Store,Pizza Place,Japanese Restaurant,Gym / Fitness Center,Gym,Yoga Studio,Café,Bookstore,Bar
2,Central Harlem,Chinese Restaurant,African Restaurant,Cosmetics Shop,Bar,Seafood Restaurant,French Restaurant,American Restaurant,Bookstore,Café,Boutique
3,Chelsea,Art Gallery,Coffee Shop,Café,Ice Cream Shop,Hotel,Market,Cupcake Shop,Cycle Studio,Theater,Italian Restaurant
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Bar,Salon / Barbershop,Coffee Shop,Optical Shop,Spa,American Restaurant,Vietnamese Restaurant


## Cluster Neighborhoods
* Clustering
  - Create New Dataframe That Includes the Cluster
  - Visualizing the Resulting Clusters
  - Examine Clusters

In [107]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state = 0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 4, 1, 4, 1, 1, 4, 1, 1], dtype=int32)

### Create New Dataframe That Includes the Cluster

In [108]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
# Toronto Wiki        = https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
# Toronto Coordinates = http://cocl.us/Geospatial_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# Display dataframe
manhattan_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant,Pizza Place
1,Manhattan,Chinatown,40.715618,-73.994279,4,Chinese Restaurant,Bakery,Cocktail Bar,Bar,Salon / Barbershop,Coffee Shop,Optical Shop,Spa,American Restaurant,Vietnamese Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Bakery,Mobile Phone Shop,Grocery Store,Deli / Bodega,Chinese Restaurant,Bank,Spanish Restaurant,Pizza Place,Latin American Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,4,Mexican Restaurant,Lounge,Restaurant,Café,Pizza Place,Wine Bar,Bakery,American Restaurant,Caribbean Restaurant,Frozen Yogurt Shop
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant,Cocktail Bar,Indian Restaurant,Sushi Restaurant,Park,Yoga Studio


### Visualizing the Resulting Clusters

In [109]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

# For Loop
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius       = 5,
        popup        = label,
        color        = rainbow[cluster-1],
        fill         = True,
        fill_color   = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)

# Display Map
map_clusters

### Examine Clusters

#### Cluster 1

In [110]:
# Cluster 1
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Tribeca,Italian Restaurant,Park,Wine Bar,Café,Spa,Hotel,Coffee Shop,Bakery,Men's Store,Steakhouse
24,West Village,Italian Restaurant,Wine Bar,American Restaurant,Coffee Shop,Park,Jazz Club,Seafood Restaurant,New American Restaurant,Pizza Place,Bakery
35,Turtle Bay,Café,Coffee Shop,Italian Restaurant,Wine Bar,Park,Deli / Bodega,Sushi Restaurant,French Restaurant,Turkish Restaurant,Seafood Restaurant
39,Hudson Yards,Italian Restaurant,Hotel,Gym / Fitness Center,American Restaurant,Restaurant,Café,Park,Dog Run,Coffee Shop,Gym


#### Cluster 2

In [111]:
# Cluster 2
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Italian Restaurant,Juice Bar,Bakery,Gym / Fitness Center,Exhibit,Hotel,Yoga Studio,Wine Shop,Spa,Mexican Restaurant
9,Yorkville,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Bar,Deli / Bodega,Japanese Restaurant,Wine Shop,Diner,Mexican Restaurant
10,Lenox Hill,Italian Restaurant,Pizza Place,Coffee Shop,Café,Sushi Restaurant,Cocktail Bar,Gym / Fitness Center,Gym,Burger Joint,Salad Place
12,Upper West Side,Italian Restaurant,Bakery,Coffee Shop,Ice Cream Shop,Bookstore,Bar,Bagel Shop,Mediterranean Restaurant,Seafood Restaurant,Sushi Restaurant
13,Lincoln Square,Italian Restaurant,Plaza,Café,Gym / Fitness Center,Theater,Performing Arts Venue,Concert Hall,American Restaurant,Wine Shop,Indie Movie Theater
14,Clinton,Theater,Gym / Fitness Center,Coffee Shop,Wine Shop,Hotel,Gym,Sandwich Place,Pizza Place,Spa,Italian Restaurant
15,Midtown,Coffee Shop,Clothing Store,Hotel,Theater,Pizza Place,Cuban Restaurant,Bakery,Sporting Goods Shop,Steakhouse,Spa
16,Murray Hill,Coffee Shop,Sandwich Place,Hotel,Gym / Fitness Center,Japanese Restaurant,Pizza Place,Steakhouse,Indian Restaurant,Bakery,Chinese Restaurant
17,Chelsea,Art Gallery,Coffee Shop,Café,Ice Cream Shop,Hotel,Market,Cupcake Shop,Cycle Studio,Theater,Italian Restaurant
18,Greenwich Village,Italian Restaurant,Coffee Shop,Bakery,Wine Bar,Pizza Place,Ice Cream Shop,Gym,Comedy Club,Sandwich Place,Restaurant


#### Cluster 3

In [112]:
# Cluster 3
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant,Pizza Place


#### Cluster 4

In [113]:
# Cluster 4
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Battery Park City,Park,Hotel,Gym,Memorial Site,Coffee Shop,Gourmet Shop,Wine Shop,Food Court,Shopping Mall,Boat or Ferry
37,Stuyvesant Town,Boat or Ferry,Park,Baseball Field,Heliport,Gas Station,German Restaurant,Bistro,Farmers Market,Gym,Gym / Fitness Center


#### Cluster 5

In [114]:
# Cluster 5
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Bar,Salon / Barbershop,Coffee Shop,Optical Shop,Spa,American Restaurant,Vietnamese Restaurant
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Grocery Store,Deli / Bodega,Chinese Restaurant,Bank,Spanish Restaurant,Pizza Place,Latin American Restaurant
3,Inwood,Mexican Restaurant,Lounge,Restaurant,Café,Pizza Place,Wine Bar,Bakery,American Restaurant,Caribbean Restaurant,Frozen Yogurt Shop
4,Hamilton Heights,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant,Cocktail Bar,Indian Restaurant,Sushi Restaurant,Park,Yoga Studio
5,Manhattanville,Seafood Restaurant,Coffee Shop,Italian Restaurant,Mexican Restaurant,Chinese Restaurant,Park,Deli / Bodega,Food Court,Climbing Gym,Boutique
6,Central Harlem,Chinese Restaurant,African Restaurant,Cosmetics Shop,Bar,Seafood Restaurant,French Restaurant,American Restaurant,Bookstore,Café,Boutique
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Latin American Restaurant,Cocktail Bar,Sandwich Place,Grocery Store,Coffee Shop,Liquor Store
11,Roosevelt Island,Coffee Shop,Residential Building (Apartment / Condo),Scenic Lookout,Sandwich Place,Metro Station,Supermarket,Liquor Store,Baseball Field,Greek Restaurant,Dry Cleaner
20,Lower East Side,Chinese Restaurant,Pharmacy,Art Gallery,Cocktail Bar,Café,Yoga Studio,Ice Cream Shop,Women's Store,Japanese Restaurant,Juice Bar
26,Morningside Heights,Park,American Restaurant,Coffee Shop,Bookstore,Deli / Bodega,Burger Joint,Tennis Court,Pizza Place,Supermarket,Café


## Results <a name="results"></a>

The goal of this analysis was to find out about the neighborhoods and their popular venues for a person planning to relocate within the city of New York.

As we analyzed the 5 clusters, we can determine a suitable neighborhood for a relocation. In fact, Cluster 2 seems to have the most venues if a person is interested in these places.

In [115]:
# Cluster 2
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Italian Restaurant,Juice Bar,Bakery,Gym / Fitness Center,Exhibit,Hotel,Yoga Studio,Wine Shop,Spa,Mexican Restaurant
9,Yorkville,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Bar,Deli / Bodega,Japanese Restaurant,Wine Shop,Diner,Mexican Restaurant
10,Lenox Hill,Italian Restaurant,Pizza Place,Coffee Shop,Café,Sushi Restaurant,Cocktail Bar,Gym / Fitness Center,Gym,Burger Joint,Salad Place
12,Upper West Side,Italian Restaurant,Bakery,Coffee Shop,Ice Cream Shop,Bookstore,Bar,Bagel Shop,Mediterranean Restaurant,Seafood Restaurant,Sushi Restaurant
13,Lincoln Square,Italian Restaurant,Plaza,Café,Gym / Fitness Center,Theater,Performing Arts Venue,Concert Hall,American Restaurant,Wine Shop,Indie Movie Theater
14,Clinton,Theater,Gym / Fitness Center,Coffee Shop,Wine Shop,Hotel,Gym,Sandwich Place,Pizza Place,Spa,Italian Restaurant
15,Midtown,Coffee Shop,Clothing Store,Hotel,Theater,Pizza Place,Cuban Restaurant,Bakery,Sporting Goods Shop,Steakhouse,Spa
16,Murray Hill,Coffee Shop,Sandwich Place,Hotel,Gym / Fitness Center,Japanese Restaurant,Pizza Place,Steakhouse,Indian Restaurant,Bakery,Chinese Restaurant
17,Chelsea,Art Gallery,Coffee Shop,Café,Ice Cream Shop,Hotel,Market,Cupcake Shop,Cycle Studio,Theater,Italian Restaurant
18,Greenwich Village,Italian Restaurant,Coffee Shop,Bakery,Wine Bar,Pizza Place,Ice Cream Shop,Gym,Comedy Club,Sandwich Place,Restaurant


## Discussion
As seen in the table above, if a person wishes to move to New York, this can be a good option for them. For instance, this cluster contains restauraunts, gyms, etc. This is just one example of how our data analysis can help people relocate from one part of the world to another. 

## Conclusion <a name="conclusion"></a>
The world we live contains data everywhere we go, where many real-life problems or scenarios can be used to find solutions to these problems. This project showed that data was used to cluster neighborhoods in New York based on the most common venues in those neighborhoods. Therefore, someone that is planning to relocate where there are shops, restaurants, and gyms can use this data as a guide.

## References
* New York City Data
  - The following Catalog Data page was scraped to pull out the necessary information: https://data.cityofnewyork.us/api/views/swpk-hqdp/rows.csv?accessType=DOWNLOAD https://geo.nyu.edu/catalog/nyu_2451_34572


* Coordinate Data for each Neighborhood
  - The following csv file was used to get the latitude and longitude for NYC's Neighborhoods: https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=Original


* Foursquare API
  - The following site was used to get latitude, longitude, and data for venues: https://foursquare.com